## Solutions - Problem 1

Get number of flights departed from each of the US airport in the month of 2008 January.

* We have to use airport codes to determine US airport.
* We need to use airtraffic data to get departure details.
* To solve this problem we have to perform inner join.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.spark.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [3]:
airtrafficPath = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [4]:
airtraffic = spark. \
    read. \
    parquet(airtrafficPath)

In [5]:
airtraffic. \
    select(
        "Year", "Month", "DayOfMonth", 
        "Origin", "Dest", "CRSDepTime"
    ). \
show()

+----+-----+----------+------+----+----------+
|Year|Month|DayOfMonth|Origin|Dest|CRSDepTime|
+----+-----+----------+------+----+----------+
|2008|    1|        16|   BGR| CVG|      1735|
|2008|    1|        17|   SYR| CVG|      1701|
|2008|    1|        17|   SAV| BOS|      1225|
|2008|    1|        17|   CVG| GRR|      1530|
|2008|    1|        17|   STL| CVG|      1205|
|2008|    1|        18|   STL| JFK|      1150|
|2008|    1|        18|   MCI| CVG|      1009|
|2008|    1|        19|   TUL| CVG|       835|
|2008|    1|        20|   JFK| PHL|      1935|
|2008|    1|        20|   RDU| CVG|       830|
|2008|    1|        21|   CVG| DTW|      1640|
|2008|    1|        21|   MSY| LGA|      1204|
|2008|    1|        21|   JFK| PHL|      1935|
|2008|    1|        21|   DCA| JFK|      1830|
|2008|    1|        21|   HSV| DCA|       700|
|2008|    1|        22|   ORD| CVG|      1910|
|2008|    1|        22|   CVG| JFK|      1320|
|2008|    1|        23|   LGA| SAV|       908|
|2008|    1| 

In [6]:
airtraffic.count()

605659

In [7]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [8]:
def getValidAirportCodes(airportCodesPath):
    airportCodes = spark. \
        read. \
        option("sep", "\t"). \
        option("header", True). \
        option("inferSchema", True). \
        csv(airportCodesPath). \
        filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    return airportCodes

In [9]:
airportCodes = getValidAirportCodes(airportCodesPath)

In [10]:
airportCodes.count()

443

In [11]:
from pyspark.sql.functions import col, lit, count

In [12]:
airtraffic. \
    join(airportCodes, airportCodes["IATA"] == airtraffic["Origin"]). \
    select(col("Year"), col("Month"), col("DayOfMonth"), airportCodes["*"], col("CRSDepTime")). \
    show()

+----+-----+----------+-------------+-----+-------+----+----------+
|Year|Month|DayOfMonth|         City|State|Country|IATA|CRSDepTime|
+----+-----+----------+-------------+-----+-------+----+----------+
|2008|    1|        16|       Bangor|   ME|    USA| BGR|      1735|
|2008|    1|        17|     Syracuse|   NY|    USA| SYR|      1701|
|2008|    1|        17|     Savannah|   GA|    USA| SAV|      1225|
|2008|    1|        17|   Cincinnati|   OH|    USA| CVG|      1530|
|2008|    1|        17|    St. Louis|   MO|    USA| STL|      1205|
|2008|    1|        18|    St. Louis|   MO|    USA| STL|      1150|
|2008|    1|        18|  Kansas City|   MO|    USA| MCI|      1009|
|2008|    1|        19|        Tulsa|   OK|    USA| TUL|       835|
|2008|    1|        20|     New York|   NY|    USA| JFK|      1935|
|2008|    1|        20|      Raleigh|   NC|    USA| RDU|       830|
|2008|    1|        21|   Cincinnati|   OH|    USA| CVG|      1640|
|2008|    1|        21|  New Orleans|   LA|    U

In [13]:
airtraffic. \
    join(airportCodes, airportCodes["IATA"] == airtraffic["Origin"]). \
    groupBy("Origin"). \
    agg(count(lit(1)).alias("FlightCount")). \
    orderBy(col("FlightCount").desc()). \
    show()

+------+-----------+
|Origin|FlightCount|
+------+-----------+
|   ATL|      33897|
|   ORD|      29936|
|   DFW|      23861|
|   DEN|      19477|
|   LAX|      18945|
|   PHX|      17695|
|   IAH|      15531|
|   LAS|      15292|
|   DTW|      14357|
|   EWR|      12467|
|   SLC|      12401|
|   MSP|      11800|
|   SFO|      11573|
|   MCO|      11070|
|   CLT|      10752|
|   LGA|      10300|
|   JFK|      10023|
|   BOS|       9717|
|   BWI|       8883|
|   CVG|       8659|
+------+-----------+
only showing top 20 rows

